# installing Jpype and Jsprit Libraries

In [ ]:
!pip install jpype
!pip install up_jsprit
!pip install unified_planning

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.5/635.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=1f748d4122e225c6b1e7ee7efe10d2a5d539dd732e7b124bbba023d36c8b

# Importing jpype to run

In [ ]:
import jpype
import jpype.imports
from jpype.types import *
import os
from up_jsprit import config #for local distibution
from up_jsprit.utils import generate_map, get_distance_and_time_from_graphhopper #for local distibution
from typing import Callable, IO, Optional
import unified_planning as up
from unified_planning import engines
import random

### import Java libraries from graphhopper

In [ ]:
env = up.environment.get_environment()
env.factory.add_engine('jspritplanner', 'up_jsprit', 'JSpritSolver')

#env = problem.environment

# Start the JVM and set classpaths
jpype.getDefaultJVMPath()

if not jpype.isJVMStarted():
    jpype.startJVM(convertStrings=False)
    print("JVM Started")
if jpype.isJVMStarted():
    print("JVM already started")

# Add the paths to the JAR files
for jar_file in config.JAR_FILES:
    jpype.addClassPath(jar_file)

#Import necessary Java From Jsprit classes using JClass
VehicleRoutingProblem = jpype.JClass("com.graphhopper.jsprit.core.problem.VehicleRoutingProblem")
VehicleImpl = jpype.JClass("com.graphhopper.jsprit.core.problem.vehicle.VehicleImpl")
VehicleType = jpype.JClass("com.graphhopper.jsprit.core.problem.vehicle.VehicleTypeImpl")
Location = jpype.JClass("com.graphhopper.jsprit.core.problem.Location")
Shipment = jpype.JClass("com.graphhopper.jsprit.core.problem.job.Shipment")
Plotter = jpype.JClass("com.graphhopper.jsprit.analysis.toolbox.Plotter")
SelectBest = jpype.JClass("com.graphhopper.jsprit.core.algorithm.selector.SelectBest")
SolutionPrinter = jpype.JClass("com.graphhopper.jsprit.core.reporting.SolutionPrinter")
GraphStreamViewer = jpype.JClass("com.graphhopper.jsprit.analysis.toolbox.GraphStreamViewer")
AlgorithmSearchProgressChartListener = jpype.JClass("com.graphhopper.jsprit.analysis.toolbox.AlgorithmSearchProgressChartListener")
Jsprit = jpype.JClass("com.graphhopper.jsprit.core.algorithm.box.Jsprit")
Coordinate = jpype.JClass("com.graphhopper.jsprit.core.util.Coordinate")
VehicleRoutingTransportCostsMatrix = jpype.JClass("com.graphhopper.jsprit.core.util.VehicleRoutingTransportCostsMatrix")
VehicleRoutingTransportCostsMatrix = jpype.JClass("com.graphhopper.jsprit.core.util.VehicleRoutingTransportCostsMatrix")
FastVehicleRoutingTransportCostsMatrix = jpype.JClass("com.graphhopper.jsprit.core.util.FastVehicleRoutingTransportCostsMatrix")
#VehicleRoutingAlgorithm = jpype.JClass("com.graphhopper.jsprit.core.problem.VehicleRoutingAlgorithm")
# Import necessary Java classes for printing the solution
PrintWriter = jpype.JClass('java.io.PrintWriter')
File = jpype.JClass('java.io.File')
PrintStream = jpype.JClass('java.io.PrintStream')
System = jpype.JClass('java.lang.System')

JVM Started
JVM already started


In [ ]:
#adding other library
Service = jpype.JClass("com.graphhopper.jsprit.core.problem.job.Service")


In [ ]:
FastVehicleRoutingTransportCostsMatrix = jpype.JClass("com.graphhopper.jsprit.core.util.FastVehicleRoutingTransportCostsMatrix")

In [ ]:
VrpXMLWriter=jpype.JClass("java.io.OutputStream")

### The attempt to build a sample simple model to run from the Java directly

In [ ]:
#STEP 0 - BUILD VEHICLE ROUTING PROBLEM

vrpbuilder = VehicleRoutingProblem.Builder.newInstance()
#Building the model of two vehicle types


WEIGHT_INDEX = 0;
vehicleTypeBuilder = VehicleType.Builder.newInstance("type").addCapacityDimension(WEIGHT_INDEX,100).setCostPerDistance(1).setCostPerTime(2).setFixedCost(2);
vehicleType1 = vehicleTypeBuilder.build();


vehicleTypeBuilder2 = VehicleType.Builder.newInstance("type").addCapacityDimension(WEIGHT_INDEX,1000).setCostPerDistance(4).setCostPerTime(4).setFixedCost(4);
vehicleType2 = vehicleTypeBuilder.build();

#type = VehicleTypeImpl.Builder.newInstance("type").addCapacityDimension(0, 1).setCostPerDistance(1).setCostPerTime(2).build();

# setting initial and final location of the vehicles
vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle1");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType1);
vehicle1 = vehicleBuilder.build();


vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle2");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType1);
vehicle2 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle3");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle3 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle4");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle4 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle5");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle5 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle6");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle6 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle7");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle7 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle8");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle8 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle9");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle9 = vehicleBuilder.build();

vehicleBuilder = VehicleImpl.Builder.newInstance("vehicle10");
vehicleBuilder.setStartLocation(Location.newInstance("0"));
vehicleBuilder.setEndLocation(Location.newInstance("0"));
vehicleBuilder.setType(vehicleType2);
vehicle10 = vehicleBuilder.build();

#STEP 1 - Adding the routing
#create a for loop for shipment 1000 locations

# Fill the Distance and Time Matrices:
for L in range(0,101):

    shipment_builder = Shipment.Builder.newInstance(str(L)).setPickupLocation(Location.newInstance("0"))
    shipment_builder.setDeliveryLocation(Location.newInstance(str(L)))
    shipment_builder.addSizeDimension(0,10);
    shipment_builder.setPickupServiceTime(1);
    shipment_builder.addPickupTimeWindow(0, 200);
    vrpbuilder.addJob(shipment_builder.build())



#s2 = Service.Builder.newInstance("2").addSizeDimension(0, 1).setLocation(Location.newInstance("2")).build();
#s3 = Service.Builder.newInstance("3").addSizeDimension(0, 1).setLocation(Location.newInstance("3")).build();
#s4 = Service.Builder.newInstance("4").addSizeDimension(0, 1).setLocation(Location.newInstance("4")).build();

#STEP 2 - BUILDING Customized cost matrix

costMatrixBuilder = VehicleRoutingTransportCostsMatrix.Builder.newInstance(True);


#creating a cost matrix using a for loop


# Fill the Distance and Time Matrices:
for i in range(0,101):
    for j in range(0,101):
        if i < j:  # Only compute for the upper triangle
            tm=random.randint(1, 10)
            ds=random.randint(1, 1000)
            costMatrixBuilder.addTransportDistance(str(i),str(j),ds )  #generate a random number for cost distance matix
            costMatrixBuilder.addTransportTime(str(i),str(j),tm)         #generate a random number for cost time matix
            print(f"{i}, {j}, {ds}, {tm}")

#costMatrixBuilder.addTransportDistance("0", "1", 10.0);
#costMatrixBuilder.addTransportDistance("0", "2", 20.0);
#costMatrixBuilder.addTransportDistance("0", "3", 5.0);
#costMatrixBuilder.addTransportDistance("1", "2", 4.0);
#costMatrixBuilder.addTransportDistance("1", "3", 1.0);
#costMatrixBuilder.addTransportDistance("2", "3", 2.0);

#costMatrixBuilder.addTransportTime("0", "1", 10.0);
#costMatrixBuilder.addTransportTime("0", "2", 20.0);
#costMatrixBuilder.addTransportTime("0", "3", 5.0);
#costMatrixBuilder.addTransportTime("1", "2", 4.0);
#costMatrixBuilder.addTransportTime("1", "3", 1.0);
#costMatrixBuilder.addTransportTime("2", "3", 2.0);


Streaming output truncated to the last 5000 lines.
0, 51, 624, 1
0, 52, 444, 7
0, 53, 681, 5
0, 54, 180, 5
0, 55, 821, 5
0, 56, 737, 2
0, 57, 282, 7
0, 58, 962, 10
0, 59, 951, 4
0, 60, 566, 5
0, 61, 762, 5
0, 62, 273, 6
0, 63, 955, 3
0, 64, 406, 3
0, 65, 317, 6
0, 66, 807, 8
0, 67, 274, 8
0, 68, 117, 6
0, 69, 326, 2
0, 70, 490, 1
0, 71, 128, 5
0, 72, 589, 2
0, 73, 256, 8
0, 74, 117, 6
0, 75, 288, 6
0, 76, 269, 6
0, 77, 421, 7
0, 78, 931, 5
0, 79, 749, 3
0, 80, 879, 5
0, 81, 814, 1
0, 82, 189, 3
0, 83, 895, 6
0, 84, 485, 3
0, 85, 532, 9
0, 86, 863, 10
0, 87, 53, 7
0, 88, 315, 8
0, 89, 532, 8
0, 90, 21, 8
0, 91, 438, 10
0, 92, 208, 1
0, 93, 328, 10
0, 94, 624, 9
0, 95, 611, 9
0, 96, 785, 1
0, 97, 892, 5
0, 98, 128, 8
0, 99, 977, 6
0, 100, 490, 2
1, 2, 616, 8
1, 3, 46, 3
1, 4, 982, 1
1, 5, 997, 6
1, 6, 782, 7
1, 7, 68, 8
1, 8, 290, 1
1, 9, 491, 1
1, 10, 90, 10
1, 11, 883, 2
1, 12, 764, 1
1, 13, 909, 7
1, 14, 343, 3
1, 15, 430, 7
1, 16, 53, 5
1, 17, 967, 5
1, 18, 348, 8
1, 19, 239, 9
1, 20

In [ ]:
# STEP 3- Run the problem and implement the solution using fast cost matrix build

costMatrix = costMatrixBuilder.build();



In [ ]:
#vrp = VehicleRoutingProblem.Builder.newInstance();
vrpbuilder.setRoutingCost(costMatrix);
vrpbuilder.addVehicle(vehicle1).addVehicle(vehicle2).addVehicle(vehicle3).addVehicle(vehicle4).addVehicle(vehicle5).addVehicle(vehicle6).addVehicle(vehicle7).addVehicle(vehicle8).addVehicle(vehicle9).addVehicle(vehicle10);


In [ ]:
vrp = vrpbuilder.build()

In [ ]:
vra = Jsprit.createAlgorithm(vrp)

In [ ]:
# step  solving the algorithm through running

# get the algorithm out-of-the-box.
algorithm = Jsprit.createAlgorithm(vrp);

# Print the solution progress as function of the iteration, added before the solution
algorithm.getAlgorithmListeners().addListener(AlgorithmSearchProgressChartListener(os.path.join(config.OUTPUT_DIRECTORY, "solution_progress.png")))

# and search a solution which returns a collection of solutions (here only one solution is constructed)
# Set the maximum number of iterations
algorithm.setMaxIterations(30)


#solve the vehicle routing algorithm
solutions = algorithm.searchSolutions();


# Retrieve the best solution
bestSolution = SelectBest().selectSolution(solutions)




#Printing the solution and the solution plot

In [ ]:
print(vrp)

[fleetSize=INFINITE][#jobs=101][#vehicles=10][#vehicleTypes=1][transportCost=com.graphhopper.jsprit.core.util.VehicleRoutingTransportCostsMatrix@29d89d5d][activityCosts=com.graphhopper.jsprit.core.problem.cost.WaitingTimeCosts@3514a4c0]


In [ ]:
print(bestSolution)

[costs=13897.0][routes=4][unassigned=0]


In [ ]:
# Create a PrintWriter that writes directly to a file
with PrintWriter(File(os.path.join(config.OUTPUT_DIRECTORY, "problemsolve.txt"))) as writer:
            SolutionPrinter.print_(writer, vrp, bestSolution, SolutionPrinter.Print.VERBOSE) # Using the corrected print_ method






In [ ]:
# Plot the solutions in order to work it needs assigned coordinates
solutionPlot = Plotter(vrp, solutions);
solutionPlot.plot(os.path.join(config.OUTPUT_DIRECTORY, "solutionPlot.png"), "solutions");

In [ ]:
# Plot the solution
solutionPlot = Plotter(vrp, bestSolution);
solutionPlot.plot(os.path.join(config.OUTPUT_DIRECTORY, "solutionPlot.png"), "Best Solution");

In [ ]:
solutions

<java object 'java.util.ArrayList'>

In [ ]:
import os
print(os.getcwd())
!ls

from google.colab import files

files.download('/content/problemsolve.txt')



/content
problemsolve.txt  sample_data


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Call the print_ method to capture its output
SolutionPrinter.print_(vrp, bestSolution, SolutionPrinter.Print.VERBOSE)  # Using the corrected print_ method

# Plot the solution
solutionPlot = Plotter(vrp, bestSolution);
solutionPlot.plot(os.path.join(config.OUTPUT_DIRECTORY, "solutionPlot.png"), "Best Solution");

In [ ]:
import os
print(os.getcwd())
!ls

from google.colab import files

files.download('/content/solution_progress.png')

/content
problemsolve.txt  sample_data  solution_progress.png


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Plot the solution
solutionPlot = Plotter(vrp, bestSolution)
solutionPlot.plot(os.path.join(config.OUTPUT_DIRECTORY, "solution_progress.png"), "Best Solution")

In [ ]:
# Plot the solution
solutionPlot = Plotter(vrp, bestSolution)
solutionPlot.plot(os.path.join(config.OUTPUT_DIRECTORY, "solutionPlot.png"), "Best Solution")

In [ ]:
# Plot the solution
solutionPlot = Plotter(vrp, bestSolution)
solutionPlot.plot(os.path.join(config.OUTPUT_DIRECTORY, "solutionPlot.png"), "Best Solution")

In [ ]:
# Stop JVM
jpype.shutdownJVM()